In [ ]:
import signac
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image

# Intro

This tutorial will cover enough to get a taste of what **signac** can do and help you apply it to your existing projects.

We will demonstrate the file-based workflow by estimating the value of pi.

We can estimate $\pi$ by placing points randomly on the square [0, 1] in x and y directions and counting how many lie within the unit circle.

![Schematic of our method](method_illustration.png)

The areas of our shapes are:

$$A_\mathrm{unit\ square} = r^2$$

$$A_\mathrm{quarter\ circle} = \frac{\pi r^2}{4}$$

and we'll use $r=1$

We'll estimate the ratio by scattering points uniformly on the square and counting how many are within the circle.

$$\frac{A_\mathrm{quarter\ circle}}{A_\mathrm{unit\ square}} = \frac{\pi}{4} \Rightarrow \pi \approx 4 * \frac{n_\mathrm{points\ in\ quarter\ circle}}{n_\mathrm{points\ in\ square}} $$



When using signac for your research, you may keep track of simulation files. In this example, we use signac's storage for our data to represent this component.

We still generate images of the work in progress and generate a summary plot.

## Outline of workshop

1. Intro to a signac project

2. Working with a signac project in a Jupyter notebook

3. Using signac-flow to automate the workflow

4. Making summary plots



# Part 1: Intro to the signac project

Ensure that the the jupyter lab file sidebar is open. 

## Create a signac project

This creates a `signac.rc` file, so signac knows where to find a project as well as an empty workspace directory.

In [ ]:
project = signac.init_project()

Create a job that eventually will estimate pi with 10 points scattered.

This job will have a state point with only 1 parameter.

In [ ]:
job = project.open_job({"num_points": 10})
job.init() # initialize the job directory

Every job gets its own job id

In [ ]:
job.id

The state point is accessible through the Job object.

In [ ]:
job.sp['num_points']

## Remember that everything is stored on the filesystem.

Observe the workspace directory (in your filesystem or juptyer sidebar) and the job directory created within it:

In [ ]:
os.listdir()

In [ ]:
os.listdir('workspace')

Note that the name of the only directory in the workspace matches the job id! Every job has its own id.

We can get this job directory with the `job.fn('')` method. This is also used to get a path to files in the job directory.

In [ ]:
job.fn('')

What files are stored in the job directory?

In [ ]:
os.listdir(job.fn(''))

So far, the job directory only contains the json file with our state point information, but we can add more soon.

# Part 2: Working with a signac project in a Jupyter notebook

You can work with signac jobs in a notebook.



## Prototyping code

Here is some code to generate random points in 2D from a specified seed.

We save the results in the job data store, which uses a binary file format called HDF5 that will be saved in the job directory.

In [ ]:
# retrieve a value from the state point
num_points = job.sp.num_points

# create a random number generator with a fixed seed so the results are replicable
rng = np.random.default_rng(2022)
points = rng.random(size=(num_points, 2))

print(points)

# save information to the job data store
job.data['points'] = points

#### **Question** What files are in the job directory now?

#### (possible solution collapsed below)

In [ ]:
# suggested solution
os.listdir(job.fn(''))

# you can also check the file system in Jupyter lab


### Access job data for a calculation

In [ ]:
with job.data:
    points = job.data.points[:]
    inside_circle = np.linalg.norm(points, axis=1) < 1
    job.data['selected'] = points[inside_circle]
    count = sum(inside_circle)

job.doc.pi_estimate = float(4 * count/num_points)

#### **Question** Notice that we stored this information in the job document. How can you access the information saved in the job document? (Hint: it's like accessing the state point)

#### (possible solution collapsed below)

In [ ]:
job.doc.pi_estimate # doc is short for "document"

#### **Exercise** This is not a very good estimate. Make a new job with a bigger value of `num_points` in its state point to use in the next section.

#### (possible solution collapsed below)

In [ ]:
# possible solution

# you can assign your job to a variable named however you want!
job = project.open_job({"num_points": 1000})
job.init()

Notice the additional directory created in the workspace.

In [ ]:
os.listdir('workspace')

#### **Question** Compare your new job id to someone else going through the tutorial. Are your job ids the same? Why or why not?

In [ ]:
my_new_job_id = job.id # saving this for later

#### (answer)

The job id will match that of another job only if the state point is identical.

### Turn previous code into functions for reuse

In [ ]:
def scatter_square(job):
    """Generate job.sp.num_points number
    of xy pairs of points within the [0,1] square 
    and save it to the job data store.
    
    Optionally, initialize the random number
    generator with the seed job.sp.seed"""
    num_points = job.sp.num_points
    
    # Use the seed if our job has one
    seed = job.sp.get("seed", None)
    # We'll add seed to our state point later
    rng = np.random.default_rng(seed)
    
    # defaults to within [0,1] in each dimension
    points = rng.random(size=(num_points, 2))
    job.data['points'] = points


In [ ]:
def calculate_pi(job):
    """Estimate pi by counting points within the unit circle.
    Points within the unit circle are saved in the job data store. 
    Set the job.doc.pi_estimate to the result.
    
    Input
    =====
    job, a signac job object"""
    with job.data:
        points = job.data.points[:]
        # within the unit circle
        inside_circle = np.linalg.norm(points, axis=1) < 1
        job.data['selected'] = points[inside_circle]
    count = sum(inside_circle)
    num_points = job.sp.num_points
    job.doc.pi_estimate = float(4 * count/num_points)

In [ ]:
def render_image(job):
    f,a = plt.subplots()
    with job.data:
        points = job.data.points[:]
        selected = job.data.selected[:]
    a.plot(*points.T, '.b')
    a.plot(*selected.T, 'or')
    a.set_aspect('equal')
    plt.setp(a,
             title = "Visualizing Pi Estimate",
             )
    plt.savefig(job.fn("preview.png"), format='png', dpi=150)
    plt.close(f)

### **Exercise**: Apply these functions one after the other to estimate pi with your new job.


Note: you can open another job by passing a string to the `id` argument of the `open_job` method.

In [ ]:
# access your job to run operations on it
job = project.open_job(id = my_new_job_id) # note that you only have to include enough to uniquely identify your job
print(job.id)

#### (possible solution collapsed below)

In [ ]:
scatter_square(job) # or whatever you called your new job

In [ ]:
calculate_pi(job)

In [ ]:
render_image(job)

We use an IPython function to display an image from a path, which we get with the `job.fn` method.

In [ ]:
Image(job.fn('preview.png'))

We can use the command line command to summarize the job(s) we've made:

In [ ]:
!signac schema

# Check your understanding and discuss with your neighbors or the slack channel (signac.io/slack-invite)

## • What happens if you run the functions `scatter_square(job)` and `calculate_pi(job)` functions again?
(perhaps by re-running those cells)

#### (answer collapsed below)

* The scattered points, and therefore the estimate of pi, are different each time

## • Why do you get a different result?

#### (answer collapsed below)

The pseudo-random number generator is initialized with a different seed each time.

NEXT: To make our workflow fully reproducible, we will add a `seed` to the job state point from now on.

## • What is preventing us from running these operations again?

#### (answer collapsed below)

Nothing we've implemented so far would prevent us from running an operation again. We would have to either

* keep track of what we run in a lab notebook
* check the job directory for evidence that the operation ran (such as the existence of `job.doc.pi_estimate`)

NEXT: we need a way to track when operations have run

## • What if `calculate_pi` is accidentally run before `scatter_square`?

#### (answer collapsed below)

You'll get an error trying to access 'points' data from the non-existing data store.

NEXT: We need a way to control the order of operations.

## **Exercise** Generating multiple jobs. Write code to create a job for the numbers in `nums_to_create` of these numbers, all for a seed of 2022.

We're adding a seed so that the estimate of pi from each job doesn't change if the workflow is run again.



In [ ]:
nums_to_create = range(20,45,5)

#### (a possible answer is collapsed below)

In [ ]:
for n in nums_to_create:
    job = project.open_job({"num_points": n,
                            "seed": 2022})
    job.init()

You will often place such initializing code in a script, for instance, in `init.py`

In Part 2, we covered
how to work with jobs in a juptyer notebook by passing job objects to functions. Next, we'll cover how to automate this using **signac-flow**.

# Part 3: Using signac-flow to automate the workflow

Open in the file `project.py` by double clicking it in the jupyter lab sidebar.

It contains *just* the decorators needed to convert our functions into a workflow with signac-flow.

**Exercise: Copy and paste the bodies of the functions from this notebook into `project.py` in the correct places.**

Check the current status of our project like this:

In [ ]:
!python project.py status

You should see something like this:
```
Using environment configuration: StandardEnvironment
Fetching status: 100%|█████████████████████████| 21/21 [00:00<00:00, 296.66it/s]
Fetching labels: 100%|██████████████████████████| 7/7 [00:00<00:00, 3283.40it/s]

Overview: 7 jobs/aggregates, 6 jobs/aggregates with eligible operations.

label             ratio
----------------  -------------------------------------------------------
points_generated  |███████████▍                            | 2/7 (28.57%)

operation/group      number of eligible jobs  submission status
-----------------  -------------------------  -------------------
scatter_square                             5  [U]: 7
render_image                               1  [U]: 7

[U]:unknown [R]:registered [I]:inactive [S]:submitted [H]:held [Q]:queued [A]:active [E]:error [GR]:group_registered [GI]:group_inactive [GS]:group_submitted [GH]:group_held [GQ]:group_queued [GA]:group_active [GE]:group_error

```

Note:
* Here, the total number of jobs is 7
* We have run all of the operations for one job, so the total of the number eligible jobs is 5.

Run one operation, using the verbose flag to print more status information.

In [ ]:
!python project.py run --num 1 --verbose

Note that the warning "WARNING:flow.project:Reached the maximum number of operations that can be executed, but there are still eligible operations." is reminding us that we told signac-flow to stop after 1 operation

Run every `render_image` operation that is ready to run

In [ ]:
!python project.py run --operation render_image --verbose

Note the blank `[]` next to the job ids with no operations eligible.

In [ ]:
!python project.py status --detailed

## Exercise: Run the rest of the operations in the project

#### (possible solution collapsed below)

In [ ]:
!python project.py run

# Part 4: Making summary plots

Before this section, run all operations in the project.

## A quick way to convert a signac project to tabular format

The default columns will consist of the job state point parameters and job document keys, but you can filter those with additional parameters of the `to_dataframe` function.

There are NaNs in the table because we have a heterogenous data schema (not all jobs have a `seed`), and when converting to a homogenous format, signac places NaNs in the gaps.

In [ ]:
project.to_dataframe()

## Aggregation by number

To visualize these results, we'll access the `job.doc.pi_estimate` of each job and plot it on one plot. The code for this is in the file `project-aggregation.py`, which includes a new function for this.

The last `@PiProject.operation` takes an argument that creates an aggregator,
`aggregator = flow.aggregator(sort_by = "num_points")`. With this, you can access data from sets of jobs.

The aggregate operation shows up in the status list

In [ ]:
!python project-aggregation.py status

In [ ]:
!python project-aggregation.py run

In [ ]:
Image('convergence.png')